Vincent Cohen

DSCI 550

2/6/26

# <center> Homework 1

## Question 1:

- How many flights were recorded for each month in the dataset? 
- And what is the average departure delay time for each month?

## Question 2:

- Which were the top 5 most common destinations from the 'JFK' airport, based on the frequency?

## Question 3: 

- Calculate the total distance traveled by each carrier. 
- Which carrier traveled the most distance overall?

## Question 4: 

- How much departure delay do I need to expect when I use AA in December? (based on average)

## Question 5: 

- Determine the top 5 busiest days (based on the number of flights). 
- Report the year, month, day, and the number of flights.